In [32]:
pip install us

  Using cached jellyfish-0.11.2-cp310-none-win_amd64.whl (208 kB)
  Attempting uninstall: jellyfish
    Found existing installation: jellyfish 0.6.1
    Uninstalling jellyfish-0.6.1:
      Successfully uninstalled jellyfish-0.6.1
Note: you may need to restart the kernel to use updated packages.


In [33]:
DC_STATEHOOD=1

In [1]:
import us
import pandas as pd
import numpy as np
import requests
import json


In [2]:
states = us.states.STATES
state_abbr = []

for state in states:
    state_abbr.append(state.abbr)

state_abbr = [name.lower() for name in state_abbr]

covid_deaths_2020_df = pd.DataFrame({'State': states, 'State Abbreviation':state_abbr})
covid_deaths_2020_df ["Deaths 2020-03-07"] = ""
covid_deaths_2020_df ["Deaths 2020-12-31"] = ""
covid_deaths_2020_df ["Total Deaths 2020"] = ""
covid_deaths_2020_df

,State,State Abbreviation,Deaths 2020-03-07,Deaths 2020-12-31,Total Deaths 2020
0,Alabama,al,,,
1,Alaska,ak,,,
2,Arizona,az,,,
3,Arkansas,ar,,,
4,California,ca,,,
5,Colorado,co,,,
6,Connecticut,ct,,,
7,Delaware,de,,,
8,Florida,fl,,,
9,Georgia,ga,,,


In [13]:
# url = "https://api.covidtracking.com/v1/states/"

# for state in state_abbr:
#     query_url = f"{url}{state}/daily.json"
#     response = requests.get(query_url).json()
    
# print(json.dumps(response, indent=4, sort_keys=True))


In [3]:
url = "https://api.covidtracking.com/v2/states/"
 
for index, row in covid_deaths_2020_df.iterrows():
    query_url = f"{url}{row['State Abbreviation']}/2020-03-07.json"
    response = requests.get(query_url).json()
    covid_deaths_2020_df.loc[index, "Deaths 2020-03-07"] = response["data"]["outcomes"]["death"]["total"]["value"]

for index, row in covid_deaths_2020_df.iterrows():
    query_url = f"{url}{row['State Abbreviation']}/2020-12-31.json"
    response = requests.get(query_url).json()
    covid_deaths_2020_df.loc[index, "Deaths 2020-12-31"] = response["data"]["outcomes"]["death"]["total"]["value"]

covid_deaths_2020_df


,State,State Abbreviation,Deaths 2020-03-07,Deaths 2020-12-31,Total Deaths 2020
0,Alabama,al,None,4827,
1,Alaska,ak,0,206,
2,Arizona,az,None,8864,
3,Arkansas,ar,None,3676,
4,California,ca,None,25386,
5,Colorado,co,None,4814,
6,Connecticut,ct,None,5995,
7,Delaware,de,None,926,
8,Florida,fl,None,21990,
9,Georgia,ga,None,10934,
